# Vehicle Sale Price Prediction

This notebook documents the full end-to-end process for predicting vehicle sale prices (`Sold_Amount`). It covers data understanding, feature engineering, modeling, evaluation, and experiment tracking with mlflow.

## 1. Data Loading

The dataset is provided as a `.rpt` file, which is a tab-delimited text export. We load it using pandas with explicit handling of missing values. A `load_rpt` function is imported from module in `data_handler.py`

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]  # notebooks → project root
sys.path.append(str(PROJECT_ROOT))

In [2]:

from data.data_handler import load_rpt

df_train = load_rpt(r'../data/DatiumTrain.rpt')
df_test = load_rpt(r'../data/DatiumTest.rpt')

df_train.head()

,Make,Model,MakeCode,FamilyCode,YearGroup,MonthGroup,SequenceNum,Description,CurrentRelease,ImportFlag,...,PrivateMax,NewPrice,Colour,Branch,SaleCategory,Sold_Date,Compliance_Date,Age_Comp_Months,KM,Sold_Amount
0,Holden,Commodore,HOLD,COMMODO,2008,0,0,VE Omega Sedan 4dr. Auto 4sp 3.6i,F,L,...,5500.0,34790.0,White,Perth (WA),Auction,2015-11-03 00:00:00.000,02/2008,93.0,227878.0,2000.0
1,Holden,Commodore,HOLD,COMMODO,1993,7,41,VR Executive Wagon 5dr. Auto 4sp 3.8i,F,L,...,2000.0,27978.0,Red,Belmore (NSW),Auction,2000-10-18 00:00:00.000,08/1993,86.0,153091.0,6800.0
2,Toyota,RAV4,TOYO,RAV4,2012,0,6,ACA33R MY12 CV Wagon 5dr Man 5sp 4x4 2.4i,F,L,...,15800.0,31990.0,040 - Glacier White (T),Sunshine (VIC),Dealer Only Auction,2014-02-05 00:00:00.000,10/2012,16.0,27374.0,22900.0
3,Holden,Commodore,HOLD,COMMODO,2007,0,11,VZ@VE MY07 Executive Wagon 5dr. Auto 4sp 3.6i,F,L,...,4800.0,35990.0,Quicksilver,Belmore (NSW),Auction,2011-01-10 00:00:00.000,01/2007,48.0,99452.0,10500.0
4,Toyota,Tarago,TOYO,TARAGO,2007,0,0,ACR50R GLi Wagon 8st 5dr Spts Auto 4sp 2.4i,F,L,...,12400.0,49490.0,Silver,Hobart (TAS),Special Fixed Price,2009-05-23 00:00:00.000,01/2007,28.0,44355.0,31320.0


## 2. Data Understanding & Quality Checks

We inspect missing values, target variable and potential data quality issues 

In [3]:
df_train.info()

missing_ratio = df_train.isnull().mean().sort_values(ascending=False)
missing_ratio.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50704 entries, 0 to 50703
Columns: 130 entries, Make to Sold_Amount
dtypes: float64(65), int64(6), object(59)
memory usage: 50.3+ MB


AltEngTorqueFrom          1.000000
QuickChargeVoltage        1.000000
QuickChargeMins           1.000000
NormalChargeMins          1.000000
NormalChargeVoltage       1.000000
KMRangeElectricEng        0.999961
TopSpeedElectricEng       0.999961
AltEngAmpHours            0.999882
ElectricEngineLocation    0.999487
FreeScheduledService      0.998107
AltEngPowerFrom           0.996923
AltEngTorqueTo            0.996766
PowerRPMFrom              0.995878
AltEngPowerTo             0.995326
Roofline                  0.989744
AltEngTorque              0.984518
AltEngDrive               0.983532
AltEngVolts               0.983512
AltEngPower               0.983512
AltEngCurrentType         0.983473
dtype: float64

In [4]:
df_train['Sold_Amount'].describe()

count     50700.000000
mean      16401.722226
std       10312.029249
min           0.000000
25%       10500.000000
50%       15000.000000
75%       20425.000000
max      317000.000000
Name: Sold_Amount, dtype: float64

In [5]:
df_test['Sold_Amount'].describe()

count     11488.000000
mean      18142.115396
std       11392.162124
min         185.000000
25%       11200.000000
50%       15750.000000
75%       22500.000000
max      165450.000000
Name: Sold_Amount, dtype: float64

The target variable seems to be highly right skewed, ranging from 185 to 160k in values. For some models it might affect model (like linear regression), due to highly biased coefficients towards high proportion values. One method can be transforming into 'natural logarithm' values, depending on the model performance

## 3. Feature Selection & null data handling

3.1 Drop some unused columns:

The following columns are explicitly excluded, as per instruction:

    AvgWholesale
    AvgRetail
    GoodWholesale
    GoodRetail
    TradeMin
    TradeMax
    PrivateMax


In [6]:
unused_cols = [
    'AvgWholesale','AvgRetail','GoodWholesale','GoodRetail',
    'TradeMin','TradeMax','PrivateMax'
]

df_train = df_train.drop(columns=[c for c in unused_cols if c in df_train.columns])
df_test = df_test.drop(columns=[c for c in unused_cols if c in df_test.columns])

3.2 From observations on the data, Columns below are to be dropped:

    Description
    VIN
    ExtraIdentification
    ModelCode
    MakeCode
    FamilyCode
    Series
    Branch
    SequenceNum
    SeriesModelYear
    DriveDescription
    DriveCode

They have thousands of unique values, and are mostly identifiers, which may not be useful for training

In [ ]:
drop_cols = [
    "VIN", "EngineNum", "Description", "ExtraIdentification",
    "ModelCode", "MakeCode", "FamilyCode",
    "Series", "Branch", "SequenceNum", "SeriesModelYear", "DriveDescription",'DriveCode',
]

df_train = df_train.drop(columns=[c for c in drop_cols if c in df_train.columns])
df_test = df_test.drop(columns=[c for c in drop_cols if c in df_test.columns])

3.3 Merge text columns that are identical

Some of the columns are just texts, and potentially can be merged into one. In this run, the 'badge' description is merged into a column called 'Car_Description'.

In [ ]:
merge_columns = ['BadgeDescription', 'BadgeSecondaryDescription', 'BadgeDescription', 'BodyStyleDescription', 'BodyConfigDescription']


new_col = 'Car_Description'

# Merge, and handle NaN with blank space
for df in [df_train, df_test]:
    df[new_col] = (
        df[merge_columns]
        .fillna("")
        .agg(" ".join, axis=1)
        .str.strip()
    )
    df = df.drop(columns=merge_columns, inplace=True)

3.4 Remove columns that have a lot of empty values or NaN (Can be optional)

In [24]:
threshold = 0.9  # Set threshold 90%
for df in [df_train]:
    # Compute the fraction of missing values per column
    missing_frac = df.isna().sum() / len(df)
    # Drop columns with missing fraction above threshold
    drop_cols = missing_frac[missing_frac > threshold].index.tolist()
    print(f"Dropping {len(drop_cols)} columns in training due to >{threshold*100}% missing:", drop_cols)
    df.drop(columns=drop_cols, inplace=True)
print(f"Dropping {len(drop_cols)} columns in test due to >{threshold*100}% missing:", drop_cols)
df_test.drop(columns=drop_cols, inplace=True)

Dropping 0 columns in training due to >90.0% missing: []
Dropping 0 columns in test due to >90.0% missing: []


3.5 Handle numeric and categorical columns - null values

Numeric columns - Use Imputations, in this example ** Median ** is used for imputation. Reason is it is more robust than Mean/Mode, in terms of outliers or heavy skewed distributions. Some methods like KNN can be used, for simplicity, median is used in this run

Categorical columns - Replace NaN with 'Unknown'

In [9]:
import pandas as pd

for df in [df_train, df_test]:
    # 1. Ensure numeric columns are numeric
    # num_cols = df.select_dtypes(include=['int64','float64','float32','int32']).columns.tolist()
    num_cols = df.select_dtypes(exclude=['object']).columns.tolist()
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # convert errors to NaN

    # 2. Fill numeric columns with median
    for col in num_cols:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

    # 3. Fill categorical columns with 'Unknown'
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    for col in cat_cols:
        df[col] = df[col].fillna('Unknown')

train_na_cols = df_train.columns[df_train.isna().any()].tolist()
test_na_cols = df_test.columns[df_test.isna().any()].tolist()

# Columns to drop (union) - due to all entries are NaN
cols_to_drop = list(set(train_na_cols) | set(test_na_cols))
df_train = df_train.drop(columns=cols_to_drop, errors='ignore')
df_test  = df_test.drop(columns=cols_to_drop, errors='ignore')

# Quick check for any remaining NaNs
print("Train NaNs remaining:", df_train.columns[df_train.isna().any()].tolist())
print("Test NaNs remaining :", df_test.columns[df_test.isna().any()].tolist())



c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of

Train NaNs remaining: []
Test NaNs remaining : []


3.6 Check for infinity or too large values (Can be optional)

In [10]:
import numpy as np
# Boolean mask of infinities
inf_mask = df_test.isin([np.inf, -np.inf])

# Columns with any infinity
cols_with_inf = inf_mask.any()
print("Columns with infinity values:")
print(cols_with_inf[cols_with_inf].index.tolist())

# Threshold for float64 (roughly 1e308)
threshold = 1e308

# Find columns with values exceeding threshold
too_large_cols = [col for col in df_test.select_dtypes(include=np.number).columns
                  if (df_test[col].abs() > threshold).any()]

print("Columns with too large values:", too_large_cols)

Columns with infinity values:
[]
Columns with too large values: []


3.7 Collapse rare categories, that appear less than 5% of the data (threshold), and label as 'Unknown' also

Columns are:

    Make
    Model
    FuelTypeDescription 
    SaleCategory"


In [12]:
high_card_cols = [
    "Make", "Model",
    "FuelTypeDescription",
    "SaleCategory"
]

min_freq = 0.05
for col in high_card_cols:
    freq = df_train[col].value_counts(normalize=True)
    rare = freq[freq < min_freq].index
    df_train[col] = df_train[col].replace(rare, "Other")
    df_test[col] = df_test[col].where(
        df_test[col].isin(df_train[col].unique()), "Other"
    )

## 4. Feature Engineering

Engineer additional features to capture vehicle age, usage, and power to weight ratio.

Feature 1: Vehicle age, estimate from age comp months, and drop YearGroup and Compliance Date

In [13]:
import numpy as np

for df in [df_train, df_test]:
    df['Vehicle_Age_Years'] = df['Age_Comp_Months'] / 12
    df = df.drop(columns=['YearGroup', 'Compliance_Date'], inplace=True)


C:\Users\kbchin\AppData\Local\Temp\ipykernel_23236\3987853089.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Vehicle_Age_Years'] = df['Age_Comp_Months'] / 12
C:\Users\kbchin\AppData\Local\Temp\ipykernel_23236\3987853089.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Vehicle_Age_Years'] = df['Age_Comp_Months'] / 12


Feature 2: Usage intensity, instead of total distance travelled. Drop 'KM' column

In [14]:
for df in [df_train, df_test]:
    df["KM_per_year"] = df["KM"] / (df['Vehicle_Age_Years'] + 0.1)
    df = df.drop(columns=['KM'], inplace=True)

C:\Users\kbchin\AppData\Local\Temp\ipykernel_23236\2112498152.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["KM_per_year"] = df["KM"] / (df['Vehicle_Age_Years'] + 0.1)
C:\Users\kbchin\AppData\Local\Temp\ipykernel_23236\2112498152.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["KM_per_year"] = df["KM"] / (df['Vehicle_Age_Years'] + 0.1)


Feature 3: Power to weight ratio, instead of raw power and weight

In [15]:
for df in [df_train, df_test]:
    df["Power_to_weight"] = df["Power"] / df["KerbWeight"]
    df = df.drop(columns=['Power', 'KerbWeight'], inplace=True)

C:\Users\kbchin\AppData\Local\Temp\ipykernel_23236\3846512596.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Power_to_weight"] = df["Power"] / df["KerbWeight"]
C:\Users\kbchin\AppData\Local\Temp\ipykernel_23236\3846512596.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Power_to_weight"] = df["Power"] / df["KerbWeight"]


Final training data columns

In [16]:
for col in df_train.columns:
    print(col)

Make
Model
MonthGroup
CurrentRelease
ImportFlag
LimitedEdition
WheelBaseConfig
Roofline
GearTypeDescription
GearLocationDescription
GearNum
DoorNum
EngineSize
EngineDescription
Cylinders
FuelTypeDescription
InductionDescription
OptionCategory
CamDescription
EngineTypeDescription
FuelCapacity
FuelDeliveryDescription
MethodOfDeliveryDescription
GrossCombinationMAss
GrossVehicleMass
WheelBase
Height
Length
Width
TareMass
PayLoad
PowerRPMFrom
PowerRPMTo
Torque
TorqueRPMFrom
TorqueRPMTo
RonRating
SeatCapacity
BuildCountryOriginDescription
ValvesCylinder
EngineCycleDescription
EngineConfigurationDescription
EngineLocation
Acceleration
FrontTyreSize
RearTyreSize
FrontRimDesc
RearRimDesc
TowingBrakes
TowingNoBrakes
WarrantyCustAssist
FreeScheduledService
WarrantyYears
WarrantyKM
FirstServiceKM
FirstServiceMonths
RegServiceMonths
AltEngEngineType
AltEngBatteryType
AltEngCurrentType
AltEngVolts
AltEngChargingMethod
AltEngPower
AltEngPowerFrom
AltEngPowerTo
AltEngTorque
AltEngTorqueTo
AltEngDrive

## 6. Data Split/Define training or test data

In [17]:
X_train = df_train.drop(columns=['Sold_Amount'])
y_train = df_train['Sold_Amount']

X_test = df_test.drop(columns=['Sold_Amount'])
y_test = df_test['Sold_Amount']

## 7. Encoding & Model training

7.0  Encoding and scale

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from src.model_trainer import ModelTrainer

# Identify numeric and categorical columns
num_cols = X_train.select_dtypes(include='number').columns.tolist()
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

# Numeric pipeline: impute NaNs with median, then scale
# In case there were still NaNs
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: impute NaNs with 'Unknown', then one-hot encode
# In case there were still NaNs
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Column transformer: apply transformers to respective columns
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Fit and transform train, transform test
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed  = preprocessor.transform(X_test)
y_train_processed = y_train
y_test_processed = y_test



2026/01/07 18:46:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/07 18:46:26 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYT

7.1 Linear Ridge Model - Baseline

Ridge regression adds a L2-penalty to conventional linear regression, so it prevents overfitting and multicollinearity

In [19]:
# 1️⃣ Initialize Ridge regression
ridge_model = Ridge(alpha=110) # 110

ridge_trainer = ModelTrainer("Ridge", ridge_model, X_train_processed, y_train_processed, X_test_processed, y_test_processed)
ridge_metrics = ridge_trainer.run()
print("Ridge:", ridge_metrics)

2026/01/07 18:48:09 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Ridge: {'RMSE': 57552608.43725826, 'R2 Test': 0.5565031864247749, 'R2 Train': 0.8110827219670886}


7.2 Random Forest - Tree based algorithm and enconding

Tree-based model can perform better than polynomial regression, as it models rule-based relationship between features 

In [ ]:

from sklearn.ensemble import RandomForestRegressor

# Random Forest
rf = RandomForestRegressor(
    n_estimators=800,
    max_depth=12,              
    min_samples_leaf=50,       
    min_samples_split=100,
    max_features=0.3,         
    random_state=42,
    n_jobs=-1
)
rf_trainer = ModelTrainer("RandomForest", rf, X_train_processed, y_train_processed, X_test_processed, y_test_processed)
rf_metrics = rf_trainer.run()
print("RandomForest:", rf_metrics)

2026/01/07 18:49:20 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


RandomForest: {'RMSE': 20747286.049241293, 'R2 Test': 0.8401227068065349, 'R2 Train': 0.8103142278683197}


7.3 LightGBM

A fast computing and memory efficient tree-based model and boosting technique to improve the search compared to Random Forest model. Can be useful for large dataset

In [21]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    random_state=42
)
lgbm_trainer = ModelTrainer("LightGBM", lgbm, X_train_processed, y_train_processed, X_test_processed, y_test_processed)
lgbm_metrics = lgbm_trainer.run()
print("LightGBM:", lgbm_metrics)

<frozen importlib._bootstrap>:912: ImportWarning: _ImportHookChainedLoader.exec_module() not found; falling back to load_module()
<frozen importlib._bootstrap>:530: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2026/01/07 19:03:21 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.213286 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9017
[LightGBM] [Info] Number of data points in the train set: 50704, number of used features: 1888
[LightGBM] [Info] Start training from score 16401.611645


c:\Users\kbchin\OneDrive\Documents\vehicle-sales-prediction\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
2026/01/07 19:07:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


LightGBM: {'RMSE': 12600330.76316419, 'R2 Test': 0.9029026364713033, 'R2 Train': 0.9404415914296298}


7.4 XGBoost

XGBoost provides more fine-grained control over regularization, thus preventing overfitting. It is also a boosting tree-based model than potentially perform better than Random Forest

In [22]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_trainer = ModelTrainer("XGBoost", xgb, X_train_processed, y_train_processed, X_test_processed, y_test_processed)
xgb_metrics = xgb_trainer.run()
print("XGBoost:", xgb_metrics)

<frozen importlib._bootstrap>:912: ImportWarning: _ImportHookChainedLoader.exec_module() not found; falling back to load_module()
<frozen importlib._bootstrap>:530: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2026/01/07 19:08:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2026/01/07 19:08:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/07 19:08:56 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: `_estimator_type` undefined.  Please use appropriate mixin to define estimator type.


XGBoost: {'RMSE': 13065301.043469952, 'R2 Test': 0.8993196044711536, 'R2 Train': 0.9409900467588352}


Compare models

In [23]:
results = pd.DataFrame.from_dict({
    'Ridge': ridge_metrics,
    'RandomForest': rf_metrics,
    'LightGBM': lgbm_metrics,
    'XGBoost': xgb_metrics
}, orient='index')

print(results)

                      RMSE   R2 Test  R2 Train
Ridge         5.755261e+07  0.556503  0.811083
RandomForest  2.074729e+07  0.840123  0.810314
LightGBM      1.260033e+07  0.902903  0.940442
XGBoost       1.306530e+07  0.899320  0.940990


Ridge regression serves as a linear baseline and highlights the limits of additive assumptions.

Random Forest helps to reduce the overfitting and seems to be able to model to non-linear relationship better than linear ridge model

LightGBM and XGBoost significantly improve performance by modeling nonlinear relationships and feature interactions common in vehicle pricing such as age, mileage, and make. LightGBM performs slightly better than XGBoost